In [1]:
import datetime
import os
import re

import pandas as pd

from multiplex_imaging_pipeline.utils import listfiles
from multiplex_imaging_pipeline.utils import get_ome_tiff_channels

In [2]:
"""
https://gist.github.com/wassname/1393c4a57cfcbf03641dbc31886123b8
"""

import unicodedata
import string

valid_filename_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
char_limit = 255

def clean_filename(filename, whitelist=valid_filename_chars, replace=' '):
    # replace spaces
    for r in replace:
        filename = filename.replace(r,'_')
    
    # keep only valid ascii chars
    cleaned_filename = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode()
    
    # keep only whitelisted chars
    cleaned_filename = ''.join(c for c in cleaned_filename if c in whitelist)
    if len(cleaned_filename)>char_limit:
        print("Warning, filename truncated because it was over {}. Filenames may no longer be unique".format(char_limit))
    return cleaned_filename[:char_limit]    


In [3]:
# omero stuff
group = 'HTAN'
project = 'CODEX'

In [4]:
initial = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/codex_tracking_v1.tsv',
                     sep='\t', index_col=0)
initial

,omero_run_name,region,Date of experiment,Name of person who performed experiment,Project,katmai qptiff filepath,storage1 qptiff filepath,katmai ome tiff filepath,Case name Specimen 1,Specimen 1 name,...,Location on slide Specimen 8,Specimen 8 bbox,Case name Specimen 9,Specimen 9 name,Location on slide Specimen 9,Specimen 9 bbox,Case name Specimen 10,Specimen 10 name,Location on slide Specimen 10,Specimen 10 bbox
Run Name,,,,,,,,,,,,,,,,,,,,,
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,NaN,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,NaN,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3,NaN,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4,NaN,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1,2024-07-23_HT891Z1,NaN,07/23/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...,NaN,/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...,HT891Z1,HT891Z1-S2H3Fp1_U14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,NaN,10/19/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1,HT339B1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,NaN,10/08/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,HT323B1,HT323B1-S1H3A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,reg_001,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,HT293N1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# initial = initial.iloc[1:]

In [6]:
dirs = [x.split('/')[5] for x in initial['katmai qptiff filepath'].to_list()]
dirs

['20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1',
 '20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2',
 '20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3',
 '20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4',
 '20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1',
 '20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1',
 '20240720_Prostate_serial_HT891Z1-S2H3Fp1-HE-6',
 '20240720_Prostate_serial_HT891Z1-S2H3Fp1-HE-6',
 '20240705_Human_BRCA_HT704B1_CODEX_3',
 '20240705_Human_BRCA_HT704B1_CODEX_1',
 '20240705_Human_BRCA_HT704B1_CODEX_5',
 '20240607_Human_CRC_Vanderbilt_S060TRF_S053TL',
 '20240607_Human_CRC_Vanderbilt_S062TLF_S109TRF',
 '20240606_Human_CRC_Vanderbilt_S074TLF_S026TL',
 '20240606_Human_CRC_Vanderbilt_S078TLF_S099TRF',
 '20240529_Human_CRC_Vanderbilt_S094TRF_S106TLF',
 '20240529_Human_CRC_Vanderbilt_S100TLF_S041TRF',
 '20240517_Human_CRC_Vanderbilt_S125T2F_S046TLF',
 '20240517_Human_CRC_Vanderbilt_S043TL_S118T2F',
 '20240516_Human_CRC_Vanderbilt_S058TLF_S003T',
 '20240516_Human_CR

In [7]:
# redo run dates
roots = [re.sub(r'^[0-9\/]*[-_]*(.*)$', r'\1', x) for x in dirs]
roots

['Prostate_serial_HT913Z1-S2H1FP1-CODEX_1',
 'Prostate_serial_HT913Z1-S2H1FP1-CODEX_2',
 'Prostate_serial_HT913Z1-S2H1FP1-CODEX_3',
 'Prostate_serial_HT913Z1-S2H1FP1-CODEX_4',
 'Prostate_serial_HT891Z1-S2H3Fp1_Codex1',
 'Prostate_serial_HT891Z1-S2H3Fp1_Codex1',
 'Prostate_serial_HT891Z1-S2H3Fp1-HE-6',
 'Prostate_serial_HT891Z1-S2H3Fp1-HE-6',
 'Human_BRCA_HT704B1_CODEX_3',
 'Human_BRCA_HT704B1_CODEX_1',
 'Human_BRCA_HT704B1_CODEX_5',
 'Human_CRC_Vanderbilt_S060TRF_S053TL',
 'Human_CRC_Vanderbilt_S062TLF_S109TRF',
 'Human_CRC_Vanderbilt_S074TLF_S026TL',
 'Human_CRC_Vanderbilt_S078TLF_S099TRF',
 'Human_CRC_Vanderbilt_S094TRF_S106TLF',
 'Human_CRC_Vanderbilt_S100TLF_S041TRF',
 'Human_CRC_Vanderbilt_S125T2F_S046TLF',
 'Human_CRC_Vanderbilt_S043TL_S118T2F',
 'Human_CRC_Vanderbilt_S058TLF_S003T',
 'Human_CRC_Vanderbilt_S040TL_S012TL',
 'Human_Bone_Marrow_MB-22_MSH_00870_A1_MB-19_00392_A1_PB-21_MSS_00164_A1_MB-22_MSH_00759_A1_MB-22_MSH_00608_A1_196855',
 'Human_CRC_S16-44514-E7U1_S16-44514-B2U

In [8]:
dates = []
for x in initial['Date of experiment']:
    month, day, year = x.split('/')
    date = datetime.date(int(year), int(month), int(day))
    date_str = str(date)
    dates.append(date_str)
dates

['2024-08-12',
 '2024-08-12',
 '2024-08-12',
 '2024-08-12',
 '2024-07-23',
 '2024-07-23',
 '2024-07-20',
 '2024-07-20',
 '2024-07-05',
 '2024-07-05',
 '2024-07-05',
 '2024-06-07',
 '2024-06-07',
 '2024-06-06',
 '2024-06-06',
 '2024-05-29',
 '2024-05-29',
 '2024-05-17',
 '2024-05-17',
 '2024-05-16',
 '2024-05-16',
 '2024-05-10',
 '2024-03-08',
 '2024-03-08',
 '2024-02-06',
 '2024-02-02',
 '2024-02-01',
 '2024-01-26',
 '2024-01-25',
 '2024-01-18',
 '2024-01-17',
 '2024-01-11',
 '2024-01-10',
 '2023-12-20',
 '2023-12-19',
 '2023-12-08',
 '2023-12-07',
 '2023-11-23',
 '2023-11-22',
 '2023-11-18',
 '2023-11-16',
 '2023-11-04',
 '2023-11-02',
 '2023-10-21',
 '2023-10-19',
 '2023-10-07',
 '2023-10-06',
 '2023-10-02',
 '2023-09-14',
 '2023-09-13',
 '2023-08-30',
 '2023-08-29',
 '2023-08-26',
 '2023-08-24',
 '2023-08-23',
 '2023-08-10',
 '2023-08-03',
 '2023-07-29',
 '2023-07-28',
 '2023-07-20',
 '2023-07-19',
 '2023-07-01',
 '2023-06-23',
 '2023-06-20',
 '2023-06-10',
 '2023-06-06',
 '2023-06-

In [9]:
run_strings = []
case_cols = [c for c in initial.columns if 'Case name' in c]
for i, row in initial.iterrows():
    row = row[case_cols]
    vals = [clean_filename(val) for val in row.to_list() if not pd.isnull(val)]
    vals = sorted(set(vals))
    run_strings.append('__'.join(vals))
run_strings

['HT913Z1',
 'HT913Z1',
 'HT913Z1',
 'HT913Z1',
 'HT891Z1',
 'HT891Z1',
 'HT891Z1',
 'HT891Z1',
 'HT704B1',
 'HT704B1',
 'HT704B1',
 'S053TL__S060TRF',
 'S062TLF__S109TRF',
 'S026TL__S074TLF',
 'S078TLF__S099TRF',
 'S094TRF__S106TLF',
 'S041TRF__S100TLF',
 'S046TLF__S125T2F',
 'S043TL__S118T2F',
 'S003T__S058TLF',
 'S012TL__S040TL',
 '196855__MB-19_00392__MB-22_MSH_00608__MB-22_MSH_00759__MB-22_MSH_00870__PB-21_MSS_00164',
 'S16-44514',
 'S13-31378',
 'HS-13-6668__HS-16-40491',
 'HS-12-30984__HS-13-18789',
 'HS-13-23034__HS-14-13304',
 'SN001H1__SN009H1__SN046H1__SN047H1',
 'SN007H1__SN010H1__SN018H1',
 'C3L-02551',
 'C3L-00982',
 'HT413C1_Th1k4A1',
 'HT413C1_Th1k4A1',
 'C3L-00970__C3N-00733__PA00002352',
 'C3L-01287__C3N-01200',
 'S123TRF',
 'S110TLF',
 '',
 'S18-25943',
 'S18-5591',
 'S18-5591',
 'HT088B1__HT378B1',
 'SN004H1__SN032H1__SN039H1',
 'S19_41530__S21_34912__S21_53957__S22_53426',
 'S21_11816__S21_44455',
 'S18_11798__S18_15142__S18_18215',
 'S18-9906',
 'HT206B1',
 'HT206

In [10]:
runs = [f'{date}_{run}' for date, run in zip(dates, run_strings)]
runs

['2024-08-12_HT913Z1',
 '2024-08-12_HT913Z1',
 '2024-08-12_HT913Z1',
 '2024-08-12_HT913Z1',
 '2024-07-23_HT891Z1',
 '2024-07-23_HT891Z1',
 '2024-07-20_HT891Z1',
 '2024-07-20_HT891Z1',
 '2024-07-05_HT704B1',
 '2024-07-05_HT704B1',
 '2024-07-05_HT704B1',
 '2024-06-07_S053TL__S060TRF',
 '2024-06-07_S062TLF__S109TRF',
 '2024-06-06_S026TL__S074TLF',
 '2024-06-06_S078TLF__S099TRF',
 '2024-05-29_S094TRF__S106TLF',
 '2024-05-29_S041TRF__S100TLF',
 '2024-05-17_S046TLF__S125T2F',
 '2024-05-17_S043TL__S118T2F',
 '2024-05-16_S003T__S058TLF',
 '2024-05-16_S012TL__S040TL',
 '2024-05-10_196855__MB-19_00392__MB-22_MSH_00608__MB-22_MSH_00759__MB-22_MSH_00870__PB-21_MSS_00164',
 '2024-03-08_S16-44514',
 '2024-03-08_S13-31378',
 '2024-02-06_HS-13-6668__HS-16-40491',
 '2024-02-02_HS-12-30984__HS-13-18789',
 '2024-02-01_HS-13-23034__HS-14-13304',
 '2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1',
 '2024-01-25_SN007H1__SN010H1__SN018H1',
 '2024-01-18_C3L-02551',
 '2024-01-17_C3L-00982',
 '2024-01-11_HT413C1_

In [11]:
counts = {}
for i, run in enumerate(runs):
    if run not in counts:
        counts[run] = 1
    else:
        count = counts[run] + 1
        counts[run] = count
        runs[i] = f'{run}_{count}'
runs

['2024-08-12_HT913Z1',
 '2024-08-12_HT913Z1_2',
 '2024-08-12_HT913Z1_3',
 '2024-08-12_HT913Z1_4',
 '2024-07-23_HT891Z1',
 '2024-07-23_HT891Z1_2',
 '2024-07-20_HT891Z1',
 '2024-07-20_HT891Z1_2',
 '2024-07-05_HT704B1',
 '2024-07-05_HT704B1_2',
 '2024-07-05_HT704B1_3',
 '2024-06-07_S053TL__S060TRF',
 '2024-06-07_S062TLF__S109TRF',
 '2024-06-06_S026TL__S074TLF',
 '2024-06-06_S078TLF__S099TRF',
 '2024-05-29_S094TRF__S106TLF',
 '2024-05-29_S041TRF__S100TLF',
 '2024-05-17_S046TLF__S125T2F',
 '2024-05-17_S043TL__S118T2F',
 '2024-05-16_S003T__S058TLF',
 '2024-05-16_S012TL__S040TL',
 '2024-05-10_196855__MB-19_00392__MB-22_MSH_00608__MB-22_MSH_00759__MB-22_MSH_00870__PB-21_MSS_00164',
 '2024-03-08_S16-44514',
 '2024-03-08_S13-31378',
 '2024-02-06_HS-13-6668__HS-16-40491',
 '2024-02-02_HS-12-30984__HS-13-18789',
 '2024-02-01_HS-13-23034__HS-14-13304',
 '2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1',
 '2024-01-25_SN007H1__SN010H1__SN018H1',
 '2024-01-18_C3L-02551',
 '2024-01-17_C3L-00982',
 '2024-

In [12]:
for run in runs: print(run)

2024-08-12_HT913Z1
2024-08-12_HT913Z1_2
2024-08-12_HT913Z1_3
2024-08-12_HT913Z1_4
2024-07-23_HT891Z1
2024-07-23_HT891Z1_2
2024-07-20_HT891Z1
2024-07-20_HT891Z1_2
2024-07-05_HT704B1
2024-07-05_HT704B1_2
2024-07-05_HT704B1_3
2024-06-07_S053TL__S060TRF
2024-06-07_S062TLF__S109TRF
2024-06-06_S026TL__S074TLF
2024-06-06_S078TLF__S099TRF
2024-05-29_S094TRF__S106TLF
2024-05-29_S041TRF__S100TLF
2024-05-17_S046TLF__S125T2F
2024-05-17_S043TL__S118T2F
2024-05-16_S003T__S058TLF
2024-05-16_S012TL__S040TL
2024-05-10_196855__MB-19_00392__MB-22_MSH_00608__MB-22_MSH_00759__MB-22_MSH_00870__PB-21_MSS_00164
2024-03-08_S16-44514
2024-03-08_S13-31378
2024-02-06_HS-13-6668__HS-16-40491
2024-02-02_HS-12-30984__HS-13-18789
2024-02-01_HS-13-23034__HS-14-13304
2024-01-26_SN001H1__SN009H1__SN046H1__SN047H1
2024-01-25_SN007H1__SN010H1__SN018H1
2024-01-18_C3L-02551
2024-01-17_C3L-00982
2024-01-11_HT413C1_Th1k4A1
2024-01-10_HT413C1_Th1k4A1
2023-12-20_C3L-00970__C3N-00733__PA00002352
2023-12-19_C3L-01287__C3N-01200
2

In [13]:
assert len(runs) == len(set(runs))

In [14]:
initial['omero_run_name'] = runs

In [15]:
initial

,omero_run_name,region,Date of experiment,Name of person who performed experiment,Project,katmai qptiff filepath,storage1 qptiff filepath,katmai ome tiff filepath,Case name Specimen 1,Specimen 1 name,...,Location on slide Specimen 8,Specimen 8 bbox,Case name Specimen 9,Specimen 9 name,Location on slide Specimen 9,Specimen 9 bbox,Case name Specimen 10,Specimen 10 name,Location on slide Specimen 10,Specimen 10 bbox
Run Name,,,,,,,,,,,,,,,,,,,,,
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3,2024-08-12_HT913Z1_3,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4,2024-08-12_HT913Z1_4,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,NaN,HT913Z1,HT913Z1-S2H1FP1_U102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1,2024-07-23_HT891Z1,NaN,07/23/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...,NaN,/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...,HT891Z1,HT891Z1-S2H3Fp1_U14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,NaN,10/19/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1,HT339B1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,NaN,10/08/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,HT323B1,HT323B1-S1H3A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,reg_001,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,HT293N1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## make generate whole slide ome.tiff commands

In [16]:
def get_platform(fp):
    root = fp.split('/')[-1]
    
    if root.split('.')[-1] == 'qptiff':
        platform = 'phenocycler'
    elif root.split('.')[-1] == 'tif':
        platform = 'codex'
    else:
        platform = 'raw'
    
    return platform

In [17]:
ome_cmds = []
omero_cmds = []
ome_fps = []

skip = [
    '/diskmnt/primary/CODEX/HTAN/20231006_Prostate_Serial_S18-9906_slide_3/S18-9906#3_Scan1.qptiff'
]
for i, row in initial.iterrows():
    qptiff = row['katmai qptiff filepath']
    if qptiff not in skip:
        assert qptiff.replace('/', '') == clean_filename(qptiff), f"{qptiff}"
    qptiff = re.sub(r'/$', r'', qptiff)

    run = row['omero_run_name']
    platform = get_platform(qptiff)

    prefix = '/'.join(qptiff.split('/')[:-1])
        
    out_fp = os.path.join(prefix, f'{run}.ome.tiff')
    ome_fps.append(out_fp)
    
    ome_cmd = f'mip make-ome --input-tif {qptiff} --platform {platform} --output-filepath {out_fp}'
    
    omero_cmd = f'omero import {out_fp} -T Project:name:{project}/Dataset:name:{run}'
    
    ome_cmds.append(ome_cmd)
    omero_cmds.append(omero_cmd)
    

In [18]:
for fp in ome_fps:
    print(fp)

/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/2024-08-12_HT913Z1.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/2024-08-12_HT913Z1_2.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3/2024-08-12_HT913Z1_3.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4/2024-08-12_HT913Z1_4.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1/2024-07-23_HT891Z1.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1/2024-07-23_HT891Z1_2.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240720_Prostate_serial_HT891Z1-S2H3Fp1-HE-6/2024-07-20_HT891Z1.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240720_Prostate_serial_HT891Z1-S2H3Fp1-HE-6/2024-07-20_HT891Z1_2.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240705_Human_BRCA_HT704B1_CODEX_3/2024-07-05_HT704B1.ome.tiff
/diskmnt/primary/CODEX/HTAN/20240705_Human_BRCA_HT704B1_CODEX_1/20

In [19]:
for cmd in ome_cmds:
    print(cmd)

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1-CODEX_1_Scan2.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/2024-08-12_HT913Z1.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1-CODEX_2_Scan2.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/2024-08-12_HT913Z1_2.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3/HT913Z1-S2H1FP1-CODEX_3_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3/2024-08-12_HT913Z1_3.ome.tiff
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4/HT913Z1-S2H1FP1-CODEX_4_Scan1.qptiff --

In [20]:
for cmd in omero_cmds:
    print(cmd)

omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/2024-08-12_HT913Z1.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/2024-08-12_HT913Z1_2.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1_2
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3/2024-08-12_HT913Z1_3.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1_3
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4/2024-08-12_HT913Z1_4.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1_4
omero import /diskmnt/primary/CODEX/HTAN/20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1/2024-07-23_HT891Z1.ome.tiff -T Project:name:CODEX/Dataset:name:2024-07-23_HT891Z1
omero import /diskmnt/primary/CODEX/HTAN/20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1/2024-07-23_HT891Z1_2.ome.tiff -T Project

In [21]:
assert len(ome_fps) == len(set(ome_fps))
assert len(ome_cmds) == len(set(ome_cmds))
assert len(omero_cmds) == len(set(omero_cmds))

In [22]:
initial['katmai ome tiff filepath'] = ome_fps

In [122]:
# # check for completion

# for fp, ome_cmd in zip(initial['katmai ome tiff filepath'], ome_cmds):
#     if not os.path.exists(fp):
#         print(ome_cmd)

In [123]:

# for fp, ome_cmd in zip(initial['katmai ome tiff filepath'], ome_cmds):
#     fs = os.stat(fp).st_size
#     print(fs, fp)

In [124]:
# # check for completion

# for fp, ome_cmd in zip(initial['katmai ome tiff filepath'], ome_cmds):
#     if os.path.exists(fp):
#         try:
#             channels = get_ome_tiff_channels(fp)
#             if not channels:
# #                 print(fp)
#                 print(ome_cmd)
#         except:
# #             print(fp)
#             print(ome_cmd)

## make generate specimen specific ome.tiff commands

In [23]:
initial

,omero_run_name,region,Date of experiment,Name of person who performed experiment,Project,katmai qptiff filepath,storage1 qptiff filepath,katmai ome tiff filepath,Case name Specimen 1,Specimen 1 name,...,Location on slide Specimen 8,Specimen 8 bbox,Case name Specimen 9,Specimen 9 name,Location on slide Specimen 9,Specimen 9 bbox,Case name Specimen 10,Specimen 10 name,Location on slide Specimen 10,Specimen 10 bbox
Run Name,,,,,,,,,,,,,,,,,,,,,
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,HT913Z1,HT913Z1-S2H1FP1_U2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,HT913Z1,HT913Z1-S2H1FP1_U51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3,2024-08-12_HT913Z1_3,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,HT913Z1,HT913Z1-S2H1FP1_U62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4,2024-08-12_HT913Z1_4,NaN,08/12/2024,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,NaN,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,HT913Z1,HT913Z1-S2H1FP1_U102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20240723_Prostate_serial_HT891Z1-S2H3Fp1_Codex1,2024-07-23_HT891Z1,NaN,07/23/2024,Xiang Li,HTAN,/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...,NaN,/diskmnt/primary/CODEX/HTAN/20240723_Prostate_...,HT891Z1,HT891Z1-S2H3Fp1_U14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,NaN,10/19/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1,HT339B1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,NaN,10/08/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,HT323B1,HT323B1-S1H3A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,reg_001,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,HT293N1-S1H3A1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
ome_cmds = []
omero_cmds = []
data = []
case_cols = [c for c in initial.columns if re.findall(r'Specimen [0-9] name', c)]
fps_counts = {}
case_cols
for i, row in initial.iterrows():
    qptiff = row['katmai qptiff filepath']
    if qptiff not in skip:
        assert qptiff.replace('/', '') == clean_filename(qptiff), f'{qptiff}'
    qptiff = re.sub(r'/$', r'', qptiff)

    run = row['omero_run_name']
    platform = get_platform(qptiff)
    
    prefix = '/'.join(qptiff.split('/')[:-1])
    
    specimens = []
    for col in case_cols:
        specimen_name = row[col]
        if not pd.isnull(specimen_name):
            specimen_name = clean_filename(specimen_name)
            specimen_number = re.sub(r'^Specimen ([0-9]+) name', r'\1', col)
            case_name = row[f'Case name Specimen {specimen_number}']

            if specimen_number != '1':
                bbox = row[f'Specimen {specimen_number} bbox']
            else:
                bbox = row[f'Specimen {specimen_number} bbox (TOP,BOTTOM,LEFT,RIGHT)']
            date = run.split('_')[0].replace('-', '')
            
            out_fp = os.path.join(prefix, f'{specimen_name}__{date}.ome.tiff')
            
            if out_fp not in fps_counts:
                fps_counts[out_fp] = 1
            else:
                fps_counts[out_fp] += 1
                count = fps_counts[out_fp]
                out_fp = os.path.join(prefix, f'{specimen_name}__{count}__{date}.ome.tiff')

            if not pd.isnull(bbox):
                ome_cmd = f'mip make-ome --input-tif {qptiff} --platform {platform} --output-filepath {out_fp} --bbox {bbox}'
            else:
                ome_cmd = f'mip make-ome --input-tif {qptiff} --platform {platform} --output-filepath {out_fp}'

            omero_cmd = f'omero import {out_fp} -T Project:name:{project}/Dataset:name:{run}'

            ome_cmds.append(ome_cmd)
            omero_cmds.append(omero_cmd)

            data.append([i, run, date, case_name, specimen_name, bbox, out_fp])
            specimens.append(specimen_name)
    if len(specimens) != len(set(specimens)):
        raise RuntimeError(f'{i}, specimen names should be unique: {specimens}')

specimen_df = pd.DataFrame(
    data,
    columns=['run_name', 'omero_run_name', 'date', 'case', 'specimen', 'bbox', 'filepath'],
)
specimen_df   

,run_name,omero_run_name,date,case,specimen,bbox,filepath
0,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U2,"562,14052,0,16950",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
1,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U12,"18424,31915,774,17324",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
2,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U22,"33663,47341,649,17012",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
3,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U41,"49777,63330,3084,19261",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
4,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,20240812,HT913Z1,HT913Z1-S2H1FP1_U51,"589,13911,404,16732",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
...,...,...,...,...,...,...,...
345,20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,20211019,HT339B1,HT339B1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...
346,20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,20211008,HT323B1,HT323B1-S1H3A4,NaN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...
347,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...
348,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1_2,20210921,HT293N1,HT293N1-S1H3A1,NaN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...


In [25]:
for cmd in ome_cmds: print(cmd)

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1-CODEX_1_Scan2.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U2__20240812.ome.tiff --bbox 562,14052,0,16950
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1-CODEX_1_Scan2.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U12__20240812.ome.tiff --bbox 18424,31915,774,17324
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1-CODEX_1_Scan2.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U22__20240812.ome.tiff --bbox 33663,47341,649,17012
mip make-ome --input-tif /disk

In [26]:
for cmd in omero_cmds: print(cmd)

omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U2__20240812.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U12__20240812.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U22__20240812.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U41__20240812.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2/HT913Z1-S2H1FP1_U51__20240812.ome.tiff -T Project:name:CODEX/Dataset:name:2024-08-12_HT913Z1_2
omero import /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CO

In [27]:
assert specimen_df.shape[0] == len(set(specimen_df['filepath']))
assert len(ome_cmds) == len(set(ome_cmds))
assert len(omero_cmds) == len(set(omero_cmds))

In [28]:
specimen_df.to_csv('/diskmnt/Projects/Users/estorrs/sandbox/codex_specimens.txt', sep='\t', index=False)

In [29]:
print('\t'.join(specimen_df.columns))
for i in range(specimen_df.shape[0]):
    print('\t'.join([x for x in specimen_df.iloc[i]]))

run_name	omero_run_name	date	case	specimen	bbox	filepath
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1	2024-08-12_HT913Z1	20240812	HT913Z1	HT913Z1-S2H1FP1_U2	562,14052,0,16950	/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U2__20240812.ome.tiff
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1	2024-08-12_HT913Z1	20240812	HT913Z1	HT913Z1-S2H1FP1_U12	18424,31915,774,17324	/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U12__20240812.ome.tiff
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1	2024-08-12_HT913Z1	20240812	HT913Z1	HT913Z1-S2H1FP1_U22	33663,47341,649,17012	/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U22__20240812.ome.tiff
20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1	2024-08-12_HT913Z1	20240812	HT913Z1	HT913Z1-S2H1FP1_U41	49777,63330,3084,19261	/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1

TypeError: sequence item 5: expected str instance, float found

In [59]:
# check for completion

for i, fp, cmd in zip(specimen_df['omero_run_name'], specimen_df['filepath'], ome_cmds):
    if not os.path.exists(fp):
#         print(i, fp)
        print(cmd)

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S058TLF_S003T/S058TLF_S003T_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S058TLF_S003T/S058TLF__20240516.ome.tiff --bbox 0,26351,0,25543
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S058TLF_S003T/S058TLF_S003T_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S058TLF_S003T/S003T__20240516.ome.tiff --bbox 42926,52890,6669,19124
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S040TL_S012TL/S040TL_S012TL_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S040TL_S012TL/S040TL__20240516.ome.tiff --bbox 0,16815,0,21429
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20240516_Human_CRC_Vanderbilt_S040TL_S012TL/S040TL_S012TL_Scan1.qptiff --platform 

mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-32775_Primary_and_Met/S16-32775_Primary_and_Met_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-32775_Primary_and_Met/mCRC_S15-32775_Primary__20230826.ome.tiff --bbox 0,43100,0,35520
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-32775_Primary_and_Met/S16-32775_Primary_and_Met_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-32775_Primary_and_Met/mCRC_S15-32775_Met__20230826.ome.tiff --bbox 48900,66960,8600,27900
mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm_Scan3.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT271B1-S1H6A5__

In [35]:
# check for completion

for fp, ome_cmd in zip(specimen_df['filepath'], ome_cmds):
    if os.path.exists(fp):
        try:
            channels = get_ome_tiff_channels(fp)
            if not channels:
#                 print(fp)
                print(ome_cmd)
        except:
#             print(fp)
            print(ome_cmd)

/diskmnt/Projects/Users/estorrs/miniconda3/envs/mip/lib/python3.8/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


mip make-ome --input-tif /diskmnt/primary/CODEX/HTAN/20230913_BRCA_HT425B1-Extra__HT591B1-S1H2__HT591B1-S1H1/HT425B1-Extra__HT591B1-S1H2__HT591B1-S1H1_Scan1.qptiff --platform phenocycler --output-filepath /diskmnt/primary/CODEX/HTAN/20230913_BRCA_HT425B1-Extra__HT591B1-S1H2__HT591B1-S1H1/HT425B1-Extra__20230913.ome.tiff --bbox 0,26000,0,14000
